## 11. heatmap tweet volume
Plot heatmap of tweet volume involving AA, sorted by weekday - hour.

**input**  :pickle file at /obj folder and database  
**output** :plots of tweet volume heatmap

In [ ]:
import sqlite3
import pandas as pd
import time 
import numpy as np
conn = sqlite3.connect('core.db',isolation_level = None)
cur = conn.cursor()

### BELOW THIS IS THE FIRST PART: PLOTTING THE WORD RELATED HEATMAP(DELAY)###

In [ ]:
# create dataframe with columns of 'created_at', 'text', 'sentiment_score', 'word' with conversation

delay = "delay"

def get_word_df(word): 
    st = time.time() 
    df_all_word = pd.DataFrame()
    query = "SELECT created_at,text,sentiment_score, user_id FROM twitter,conversation WHERE twitter.id = conversation.tweet_id AND text LIKE '%"+ word +"%'"
    df = pd.read_sql_query(query,conn)
    df['word'] = word
    df_all_word = df_all_word.append(df,ignore_index = True)
    return df_all_word 


df_delay = get_word_df(delay)

In [ ]:
airlines = [56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406,
            38676903, 1542862735, 253340062, 218730857, 45621423, 20626359] 

weekdays = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

def get_heatmap_val(df):
    
    heatmap_df = pd.DataFrame(index = np.arange(0,24), columns = weekdays)   
    selected = df.loc[~df['user_id'].isin(airlines)]  
    for string in selected['created_at']: 
        weekday = string[0:3]
        hour = int(string[11:13])
        if format(heatmap_df.loc[hour, weekday]) == 'nan':
            heatmap_df.loc[hour, weekday] = 0
        heatmap_df.loc[hour, weekday] += 1
        heatmap_df = heatmap_df.astype(np.float64)
    return heatmap_df

df_delay_h = get_heatmap_val(df_delay)


In [ ]:
# import matplotlib.pyplot as plt; plt.rcdefaults()
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns 


plt.figure(figsize = (12, 8))
with sns.axes_style("white"):
    sns.heatmap(df_delay_h, cmap = "inferno_r", mask = df_delay_h.isnull())

plt.show()

### BELOW THIS ARE THE POSITIVE AND NEGATIVE HEATMAP PLOTS###

In [ ]:
import pickle
'''
This way you can load the pickle file into python.

It contains our beloveth conversations with super badass scores.
'''

conversations_with_scores = pickle.load(open("obj/conversations_with_scores.p", "rb"))

In [ ]:
from collections import defaultdict as dfd
import numpy as np

airlines = [56377143, 106062176, 18332190, 22536055, 124476322, 26223583, 2182373406,
            38676903, 1542862735, 253340062, 218730857, 45621423, 20626359]


weekday_sentiment_dict_am = dfd(list)

weekdays = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']


sen_k_df = pd.DataFrame(index = np.arange(0,24), columns = weekdays) 
sen_g_df = pd.DataFrame(index = np.arange(0,24), columns = weekdays) 

for conv in conversations_with_scores:
    id_lst = [tweet[1] for tweet in conv]
    if 22536055 in id_lst:
        cus_tweets = []
        for tweet in conv:
            if tweet[1] not in airlines: 
                cus_tweets.append(tweet)
        weekday = conv[-1][2][0:3]
        hour = int(conv[-1][2][10:13])
        if format(sen_k_df.loc[hour, weekday]) == 'nan':
            sen_k_df.loc[hour, weekday] = 0 
        if format(sen_g_df.loc[hour, weekday]) == 'nan':
            sen_g_df.loc[hour, weekday] = 0  
        if cus_tweets[-1][4] > 0: 
             sen_k_df.loc[hour, weekday] += 1 
        if cus_tweets[-1][4] < 0:
             sen_g_df.loc[hour, weekday] += 1
        

sen_k_df = sen_k_df.astype(np.float64)

sen_g_df = sen_g_df.astype(np.float64)

### below's the positive heatmap

In [ ]:
print(sen_g_df)

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns 


plt.figure(figsize = (12, 8))
with sns.axes_style("white"):
    sns.heatmap(sen_k_df, cmap = "inferno_r", mask = sen_k_df.isnull())

plt.show()

### below's the negative heatmap 

In [ ]:
import matplotlib.pyplot as plt; plt.rcdefaults()
import seaborn as sns 


plt.figure(figsize = (12, 8))
with sns.axes_style("white"):
    sns.heatmap(sen_g_df, cmap = "inferno_r", mask = sen_g_df.isnull())

plt.show()

In [ ]:
print('Done')